In [1]:
library('data.table')
library('tidyverse')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   0.3.5
✔ tibble  3.2.1     ✔ dplyr   1.1.4
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.4     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()


In [2]:
setwd('~/dpetrov/MarkB/Orchard2021Data/snptables/Orchard2021/')

In [4]:
#TROUBLESHOOT WITH 8X sync file
#Subset sync file to orch 2021 sites
setwd('~/dpetrov/MarkB/Orchard2021Data/18_HAFPipe_Validation/Bams.08x.A/')
sync = fread('test.sync')
head(sync)

V1,V2,V3,V4,V5
<chr>,<int>,<chr>,<chr>,<chr>
2L,29,A,2:0:0:0:0:0,1:0:0:0:0:0
2L,30,G,0:0:0:2:0:0,0:0:0:1:0:0
2L,31,A,2:0:0:0:0:0,1:0:0:0:0:0
2L,32,T,0:2:0:0:0:0,0:1:0:0:0:0
2L,33,A,2:0:0:0:0:0,1:0:0:0:0:0
2L,34,T,0:2:0:0:0:0,0:1:0:0:0:0


In [5]:
sync = sync %>% filter(!V1 %in% c('dmel_mitochondrion_genome', 'YHet', 'gi|42519920|ref|NC_002978.6|'))

In [3]:
#Subset sync file to orch 2021 sites
setwd('~/dpetrov/MarkB/Orchard2021Data/18_HAFPipe_Validation/Bams.100x/')
sync = fread('hi.cov.sync')

In [4]:
sync = sync %>% filter(!V1 %in% c('dmel_mitochondrion_genome', 'YHet', 'gi|42519920|ref|NC_002978.6|'))

In [5]:
load('../../RData/orch2021_Baseline_Downsampled_Filtered_Mean.RData', verbose = TRUE)
names(sync)[1:2] = c('chrom', 'pos')
sync = left_join(sites.base, sync)

Loading objects:
  eec.base.mean
  sites.base
  afmat.base.mean
  inv.recomb


Joining with `by = join_by(chrom, pos)`


In [7]:
sites.sync = sync %>% dplyr::select(chrom, pos)
afs.sync = as.data.frame(sync[, 4:ncol(sync)])

In [26]:
#Get ref allele state
files = list.files(pattern = '.snpTable$')
data = data.frame()
for (f in files){
    df = read.csv(f)
    chr = strsplit(f, '[.]')[[1]][3]
    df = df[,1:2]
    df$chrom = chr
    names(df) = c('pos', 'Ref', 'chrom')
    df = df %>% dplyr::select(chrom, pos, Ref)
    data = rbind(data, df)
    
}
write.table(data, 'orch21.sites.RefAlleles.txt', sep = '\t', quote = FALSE, row.names = FALSE)

In [9]:
#Add ref alleles to sync file

In [12]:
df = fread('../../snptables/Orchard2021/orch21.sites.RefAlleles.txt')
head(df)

chrom,pos,Ref
<chr>,<int>,<chr>
2L,5390,T
2L,5399,G
2L,5429,A
2L,5465,C
2L,5495,A
2L,5598,C


In [13]:
sites.sync = left_join(sites.sync, df)

Joining with `by = join_by(chrom, pos)`


In [14]:
sync.new = cbind(sites.sync, afs.sync)

In [15]:
head(sync.new)

,chrom,pos,Ref,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2L,5390,T,24:29:0:0:0:0,45:52:0:0:0:0,38:42:0:0:0:0,31:51:0:0:0:0,28:56:0:0:0:0,36:36:0:0:0:0,33:38:0:0:0:0,24:46:0:0:0:0,31:40:0:0:0:0,38:37:0:0:0:0,34:46:0:0:0:0,28:44:0:0:0:0,36:49:0:0:0:0,25:41:0:0:0:0,37:47:0:0:0:0,21:39:0:0:0:0
2,2L,5399,G,0:0:0:55:0:0,0:0:0:99:0:0,0:0:1:72:0:0,0:0:1:79:0:0,0:0:3:76:0:0,0:0:0:67:0:0,0:0:1:67:0:0,0:0:0:65:0:0,0:0:1:70:0:0,0:0:1:68:0:0,0:0:0:78:0:0,0:0:2:68:0:0,0:0:0:82:0:0,0:0:0:62:0:0,0:0:0:81:0:0,0:0:0:52:0:0
3,2L,5465,C,8:0:34:0:0:0,14:0:84:0:0:0,14:0:47:0:0:0,12:0:65:0:0:0,10:0:59:0:0:0,14:0:48:0:0:0,10:0:53:0:0:0,8:0:60:0:0:0,9:0:59:0:0:0,12:0:50:0:0:0,14:0:52:0:0:0,19:0:44:0:0:0,13:0:59:0:0:0,6:0:50:0:0:0,12:0:64:0:0:0,11:0:43:0:0:0
4,2L,5495,A,47:1:0:0:0:0,100:1:0:0:0:0,62:2:0:0:0:0,76:0:0:0:0:0,64:1:0:0:0:0,67:0:0:0:0:0,50:0:0:0:0:0,69:1:0:0:0:0,70:1:0:0:0:0,57:0:0:0:0:0,65:1:0:0:0:0,55:1:0:0:0:0,62:0:0:0:0:0,55:1:0:0:0:0,68:0:0:0:0:0,74:5:0:0:0:0
5,2L,5598,C,0:0:49:9:0:0,0:0:75:12:0:0,0:0:54:14:0:0,0:0:61:13:0:0,0:0:54:10:0:0,0:0:50:5:0:0,0:0:49:14:0:0,0:0:47:11:0:0,0:0:52:6:0:0,0:0:48:10:0:0,0:0:61:11:0:0,0:0:49:9:0:0,0:0:57:11:0:0,0:0:56:7:0:0,0:0:60:8:0:0,1:0:75:17:0:0
6,2L,5762,T,0:30:27:0:0:0,0:37:33:0:0:0,0:16:31:0:0:0,0:34:44:0:0:0,0:31:35:0:0:0,0:35:23:0:0:0,0:32:35:0:0:0,0:36:38:0:0:0,0:32:36:0:0:0,0:29:29:0:0:0,0:31:19:0:0:0,0:46:10:0:0:0,0:36:31:0:0:0,0:48:19:0:0:0,0:38:27:0:0:0,0:42:47:0:0:0


In [17]:
write.table(sync.new, './hi.cov.sync.orch21.txt', quote = FALSE,  row.names = FALSE)

In [18]:
test.sync = fread('../popoolation2helper/test.sync')

In [19]:
test.sync

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2L,1524,T,0:200:0:2:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0
2L,1669,G,2:0:0:200:0:0,3:0:0:199:0:0,31:0:0:171:0:0,23:0:0:179:0:0,15:0:0:187:0:0,9:0:0:193:0:0,0:0:0:202:0:0,0:0:0:202:0:0,0:0:0:202:0:0,0:0:0:202:0:0,0:0:0:202:0:0
2L,1675,A,200:0:0:2:0:0,202:0:0:0:0:0,202:0:0:0:0:0,202:0:0:0:0:0,202:0:0:0:0:0,202:0:0:0:0:0,202:0:0:0:0:0,202:0:0:0:0:0,202:0:0:0:0:0,202:0:0:0:0:0,202:0:0:0:0:0
2L,2821,C,0:2:200:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0
2L,2965,T,0:200:0:2:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0
2L,2980,G,2:0:0:200:0:0,0:0:0:202:0:0,0:0:0:202:0:0,0:0:0:202:0:0,0:0:0:202:0:0,0:0:0:202:0:0,0:0:0:202:0:0,0:0:0:202:0:0,0:0:0:202:0:0,0:0:0:202:0:0,0:0:0:202:0:0
2L,3066,T,0:200:0:2:0:0,0:197:0:5:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0
2L,3067,T,0:200:2:0:0:0,0:197:5:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0,0:202:0:0:0:0
2L,3068,C,2:0:200:0:0:0,5:0:197:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0,0:0:202:0:0:0


In [20]:
sync.new = fread('./hi.cov.sync.orch21.txt')
head(sync.new)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2L,5390,T,24:29:0:0:0:0,45:52:0:0:0:0,38:42:0:0:0:0,31:51:0:0:0:0,28:56:0:0:0:0,36:36:0:0:0:0,33:38:0:0:0:0,24:46:0:0:0:0,31:40:0:0:0:0,38:37:0:0:0:0,34:46:0:0:0:0,28:44:0:0:0:0,36:49:0:0:0:0,25:41:0:0:0:0,37:47:0:0:0:0,21:39:0:0:0:0
2L,5399,G,0:0:0:55:0:0,0:0:0:99:0:0,0:0:1:72:0:0,0:0:1:79:0:0,0:0:3:76:0:0,0:0:0:67:0:0,0:0:1:67:0:0,0:0:0:65:0:0,0:0:1:70:0:0,0:0:1:68:0:0,0:0:0:78:0:0,0:0:2:68:0:0,0:0:0:82:0:0,0:0:0:62:0:0,0:0:0:81:0:0,0:0:0:52:0:0
2L,5465,C,8:0:34:0:0:0,14:0:84:0:0:0,14:0:47:0:0:0,12:0:65:0:0:0,10:0:59:0:0:0,14:0:48:0:0:0,10:0:53:0:0:0,8:0:60:0:0:0,9:0:59:0:0:0,12:0:50:0:0:0,14:0:52:0:0:0,19:0:44:0:0:0,13:0:59:0:0:0,6:0:50:0:0:0,12:0:64:0:0:0,11:0:43:0:0:0
2L,5495,A,47:1:0:0:0:0,100:1:0:0:0:0,62:2:0:0:0:0,76:0:0:0:0:0,64:1:0:0:0:0,67:0:0:0:0:0,50:0:0:0:0:0,69:1:0:0:0:0,70:1:0:0:0:0,57:0:0:0:0:0,65:1:0:0:0:0,55:1:0:0:0:0,62:0:0:0:0:0,55:1:0:0:0:0,68:0:0:0:0:0,74:5:0:0:0:0
2L,5598,C,0:0:49:9:0:0,0:0:75:12:0:0,0:0:54:14:0:0,0:0:61:13:0:0,0:0:54:10:0:0,0:0:50:5:0:0,0:0:49:14:0:0,0:0:47:11:0:0,0:0:52:6:0:0,0:0:48:10:0:0,0:0:61:11:0:0,0:0:49:9:0:0,0:0:57:11:0:0,0:0:56:7:0:0,0:0:60:8:0:0,1:0:75:17:0:0
2L,5762,T,0:30:27:0:0:0,0:37:33:0:0:0,0:16:31:0:0:0,0:34:44:0:0:0,0:31:35:0:0:0,0:35:23:0:0:0,0:32:35:0:0:0,0:36:38:0:0:0,0:32:36:0:0:0,0:29:29:0:0:0,0:31:19:0:0:0,0:46:10:0:0:0,0:36:31:0:0:0,0:48:19:0:0:0,0:38:27:0:0:0,0:42:47:0:0:0


In [21]:
sync.sub = sync.new[1:10,]

In [23]:
write.table(sync.sub, './test.orch21.sync', quote = FALSE,  row.names = FALSE)